<a href="https://colab.research.google.com/github/Moruuu/STAMP/blob/master/UkiyoeRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

関数設定

In [0]:
!pip install hyperdash
from hyperdash import monitor_cell
!hyperdash signup --github

In [0]:
from tensorflow.keras.callbacks import Callback
from hyperdash import Experiment

class Hyperdash(Callback):
    def __init__(self, entries, exp):
        super(Hyperdash, self).__init__()
        self.entries = entries
        self.exp = exp

    def on_epoch_end(self, epoch, logs=None):
        for entry in self.entries:
            log = logs.get(entry)            
            if log is not None:
                self.exp.metric(entry, log)

exp = Experiment("UkiyoeRecognition")
hd_callback = Hyperdash(["val_loss", "loss", "val_accuracy", "accuracy"], exp)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from pathlib import Path
# 変数dataset_pathの部分は各自データを保存したパスを入れて下さい
dataset_path = Path("/content/drive/My Drive/Colab Notebooks/UkiyoeRecogition")
train_image_path = dataset_path / "ukiyoe-train-imgs.npz"
test_image_path = dataset_path / "ukiyoe-test-imgs.npz"
train_label_path = dataset_path / "ukiyoe-train-labels.npz"

In [0]:
# このセルを実行して保存したnpzファイルが参照できているか確認してください
list(dataset_path.glob("*"))

[PosixPath('/content/drive/My Drive/Colab Notebooks/UkiyoeRecogition/ukiyoe-train-labels.npz'),
 PosixPath('/content/drive/My Drive/Colab Notebooks/UkiyoeRecogition/ukiyoe-train-imgs.npz'),
 PosixPath('/content/drive/My Drive/Colab Notebooks/UkiyoeRecogition/ukiyoe-test-imgs.npz')]

In [0]:
import keras
import os
import random 
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import optimizers

In [0]:
# seed set
# reference 
# https://qiita.com/okotaku/items/8d682a11d8f2370684c9
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
random.seed(7)
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
batch_size = 128
num_classes = 10
epochs = 1000

# input image dimensions
img_rows, img_cols = 224, 224


def load(f):
    return np.load(f)['arr_0']


# Load the data
x_train = load(train_image_path)
x_test = load(test_image_path)
y_train = load(train_label_path)
# y_test = load('kmnist-test-labels.npz')

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
from keras.applications.xception import Xception
input_tensor=Input(shape=(img_cols, img_cols, 3))
num_classes=10
# Model
top_model = Sequential()
xception = Xception(include_top=False, weights=None,input_tensor=input_tensor)
# Add convolution 2D
top_model.add(Flatten(input_shape=xception.output_shape[1:]))
# 出力層
top_model.add(Dense(num_classes, activation='sigmoid'))
# Xceptionと上記モデルを結合
xception_model = Model(input=xception.input, output=top_model(xception.output))

In [0]:
xception_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])

In [0]:
history = xception_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_split=0.1)

ResourceExhaustedError: ignored

In [0]:
predicts = np.argmax(model.predict(x_test), axis=1)

In [0]:
predicts.shape

In [0]:
predicts

In [0]:
import matplotlib.pyplot as plt
# reference
# https://keras.io/ja/visualization/

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
submit = pd.DataFrame(data={"id": [], "y": []})

In [0]:
submit.id = list(range(1, predicts.shape[0]+1))

In [0]:
submit.y = predicts

In [0]:
submit.head()

In [0]:
submit.to_csv("submit.csv", index=False)

In [0]:
from google.colab import files
files.download("submit.csv")